In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

In [2]:
train = pd.read_csv('./input/train.csv')
train['pos'] = train.annotations != '[]'

In [3]:
!mkdir -p ./yolo_data/train/images
!mkdir -p ./yolo_data/train/labels    

In [4]:
for idx, x in train.iterrows():
    #if not x.pos: continue
    copyfile(f'./input/train_images/video_{x.video_id}/{x.video_frame}.jpg',
            f'./yolo_data/train/images/{x.image_id}.jpg')
    anno = eval(x.annotations)
    r = ''
    for an in anno:
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                      (an['y'] + an['height'] / 2) / 720,
                                      an['width'] / 1280,
                                      an['height'] / 720)
        
    with open(f'./yolo_data/train/labels/{x.image_id}.txt', 'a') as fp:
        fp.write(r)


In [50]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [62]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /home/xyb/Project/Kaggle/GreatBarrierReef/yolo_data/  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: val/images  # val images (relative to 'path') 128 images
#test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [63]:
%cd ~/Project/Git/yolov5
!ls -F

/home/xyb/Project/Git/yolov5
CONTRIBUTING.md  export.py   __pycache__/	setup.cfg	val.py
data/		 hubconf.py  README.md		train.py
detect.py	 LICENSE     requirements.txt*	tutorial.ipynb
Dockerfile	 models/     runs/		utils/


In [64]:
print(os.getcwd())
with open('./data/GBR_Starfish_Detection.yaml', 'a') as fp:
    fp.write(data)
with open('./data/hyps/GBR_Startfish_Hpy.yaml', 'a') as fp:
    fp.write(hyps)

/home/xyb/Project/Git/yolov5


In [61]:
!python train.py --img 3000 --batch 4 --epochs 5 --data data/GBR_Starfish_Detection.yaml --weights yolov5s6.pt --name GBR-Starfish-Detect --hyp data/hyps/GBR_Startfish_Hpy.yaml

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s6.pt, cfg=, data=data/GBR_Starfish_Detection.yaml, hyp=data/hyps/GBR_Startfish_Hpy.yaml, epochs=5, batch_size=4, imgsz=3000, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=GBR-Starfish-Detect, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-197-g0cf932b torch 1.8.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentu